In [1]:
from __future__ import print_function, division
import requests
requests.__path__

['/Users/sean/anaconda3/envs/metis/lib/python3.7/site-packages/requests']

In [2]:
url = 'https://www.boats.com/boats-for-sale/?boat-type=sail&class=sail-cruiser&length=40-50ft'
response = requests.get(url)
response.status_code

200

In [3]:
page = response.text

In [4]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [5]:
#print(soup.prettify())

Grab all the listsing in the list on this page.  Then grab the html list items

In [6]:
all_list_items = soup.find(id='listings-srp').find_all('li')

Iterate through the listings colleciton keep the actual boat listing items

In [7]:
print(all_list_items[10].attrs)

{'data-listing-id': '5637501'}


In [8]:
if 'data-listing-id' in all_list_items[10].attrs:
    print(True)

True


In [9]:
all_list_items[10].has_attr('data-listing-id')

True

In [21]:
all_list_items[10].find('a').attrs

{'href': '/sailing-boats/2020-bali-4-5-5637501/',
 'data-reporting-click-product-id': '5637501',
 'data-reporting-click-listing-type': 'standard listing'}

In [25]:
print(all_list_items[10].find('a')['href'])

/sailing-boats/2020-bali-4-5-5637501/


In [133]:
import pandas as pd
import requests
import sys
import os.path
import pickle
from bs4 import BeautifulSoup

# Initialize variables
url = 'https://www.boats.com/boats-for-sale/?boat-type=sail&class=sail-cruiser&length=40-50ft&page='
pagination_page_path = './data/pagination_page.pickle'
All_Listings_path = './data/All_Listings.pkl'
columns = ['ID','link', 'make_model', 'year', 'price', 'seller', 'location']

# Determine the pagination page and consequently the full url
if (not os.path.isfile(pagination_page_path)) or (not os.path.isfile(All_Listings_path)):
    # One or both data files don't exist so start from scratch (page 1).
    pagination_page = 1
    All_Listings = pd.DataFrame(columns=columns)
    with open(pagination_page_path, 'wb') as f:
        # Pickle the 'pagination_page' using the highest protocol available.
        pickle.dump(pagination_page, f, pickle.HIGHEST_PROTOCOL)
    print('Saved initial boats.com pagination page number: 1')
else:
    # TODO: Should implement a try/catch here in case file is corrupt.
    # Load boats.com pagination page number and previously scraped boat data.
    with open(pagination_page_path, 'rb') as f:
        # The protocol version used is detected automatically, so we do not
        # have to specify it.
        pagination_page = pickle.load(f)
    All_Listings = pd.read_pickle(All_Listings_path)
    print('Read current boats.com pagination page:', pagination_page, 
          'Starting with:', len(All_Listings), 'boats.')
url = url + str(pagination_page)

# TODO: Implement looping through 203 pagination pages
# Get webpage
# TODO: It looks like boats.com just sends you to a different page if you use an out of range page #.
# TODO: Might be fancy to implement selenium "next" rather than fixed loop
# TODO: remove comment to get url.
#response = requests.get(url)
if response.status_code != 200:
    sys.exit(response.status_code, ' Error for:', url)
page = response.text
# Engage BeautifulSoup and a selected parser
soup = BeautifulSoup(page, "lxml")

# Extract the part of the DOM that contains the list of boats
all_list_items = soup.find(id='listings-srp').find_all('li')
# Loop through the list items to find the actual listings
boats = []
for list_item in all_list_items:
    # Actual listing contain this attribute, others are advertisements
    if list_item.has_attr('data-listing-id'):
        # Extract the pieces of information contained in each list item
        ID = list_item['data-listing-id']
        link = list_item.find('a')['href']
        make_model = list_item.find('h2').string.strip()
        year = list_item.find(class_='year').string.strip()
        price = list_item.find(class_='price').string.strip().replace('$','').replace(',','')
        seller = list_item.find(class_='seller').text.replace('Seller ','').strip()
        location = list_item.find(class_='country').text.strip().title()
        boats.append([ID, link, make_model, year, price, seller, location])

All_Listings = All_Listings.append(pd.DataFrame(boats, columns=columns), ignore_index=True)
All_Listings.to_pickle(All_Listings_path)
pagination_page += 1
with open(pagination_page_path, 'wb') as f:
    # Pickle the 'pagination_page' using the highest protocol available.
    pickle.dump(pagination_page, f, pickle.HIGHEST_PROTOCOL)
print('Obtained and processed Boats.com page', str(int(pagination_page)-1).strip(), 'and stored data:',
      len(All_Listings),'boats scraped.')
url = url.replace('page=1','page=' + str(pagination_page))

All_Listings

Saved initial boats.com pagination page number: 1
Obtained and processed Boats.com page 1 and stored data: 9 boats scraped.


,ID,link,make_model,year,price,seller,location
0,5637501,/sailing-boats/2020-bali-4-5-5637501/,Bali 4.5,2020,Request Price,"Cruising Yachts Unlimited, Inc","All Locations, California"
1,7191853,/sailing-boats/2018-jeanneau-sun-odyssey-419-7...,Jeanneau Sun Odyssey 419,2018,265000,Bluenose Yacht Sales- Newport,"Portsmouth, Rhode Island"
2,7213466,/sailing-boats/1996-beneteau-oc-400-7213466/,Beneteau OC 400,1996,89500,Sail Place Inc Kenosha Office,"Kenosha , Wisconsin"
3,7216014,/sailing-boats/2002-hunter-466-7216014/,Hunter 466,2002,125900,"Latitude Yacht Brokerage, LLC","Stamford, Connecticut"
4,7152732,/sailing-boats/2012-fountaine-pajot-orana-44-7...,Fountaine Pajot Orana 44,2012,388900,POP Yachts,"St Thomas, United States"
5,7137823,/sailing-boats/2010-jeanneau-50ds-7137823/,Jeanneau 50DS,2010,289000,Bluenose Yacht Sales- Newport,"Portsmouth, Rhode Island"
6,7004954,/sailing-boats/2005-catalina-400-7004954/,Catalina 400,2005,165000,Massey Yacht Sales & Service,"Venice, Florida"
7,6824490,/sailing-boats/1979-nautor-swan-441-6824490/,Nautor Swan 441,1979,99000,"Berthon USA (Newport, Rhode Island)","Jamestown, Rhode Island"
8,6624760,/sailing-boats/1975-monk-cruiser-6624760/,Monk Cruiser,1975,89900,Yacht Connection,"Wilmington, California"


In [131]:
all_list_items[10].find(class_='country').text.strip()

'All Locations, California'

0
1


In [58]:
all_list_items[10]['data-listing-id']

'5637501'